In [ ]:
import serial
import time
import codecs
import os

def fixlat(data):
    lat_int=float(data[0:2])
    lat_float=float(data[2:])/60
    return lat_int+lat_float

def fixlng(data):
    lng_int=float(data[0:3])
    lng_float=float(data[3:])/60
    return lng_int+lng_float

def initialize():
# initialise serial port settings
    Port = serial.Serial()
    Port.baudrate = 9600#115200
    Port.port = '/dev/ttyS0'

    Port.xonxoff = 1
# return the port as an object we can use
    return Port

def getdata(port):
# open the serial port
    port.open()
# check that the port is open
    if port.isOpen():
# read 16 line
       while True:
        line=str(port.readline())
        if line.find('RMC')>0:
            if line.find(',A,')>0:
                lat=line[line.index('A')+1:line.index('N',10)].strip(',')
                lng=line[line.index('N',10)+1:line.index('E')].strip(',')
                flng = fixlng(lng)
                flat = fixlat(lat)
                break
# close the serial port
    port.close()
# discard the first line (sometimes it contains rubbish, so just always discard it)
    #del line[0]
# return the list of lines
    return flng, flat

#Ping the server and read the reponse to file f
def ping(ser):
    while True:
        ser.write(bytes("AT+NPING=120.126.136.17\r\n", 'UTF-8'))
        f = open('/home/bigdata/Desktop/nbiotWithDongle/logs', 'a+')#open logs
        response = []
        for _ in range(5):
            response.append(ser.readline().decode("utf-8"))
        
        f.write("    PING:" + str(response))
        f.close()
        if 'OK\r\n' in response:
            return
        
if __name__ == '__main__':
    ser=serial.Serial('/dev/ttyXRUSB1', 9600, timeout = 0.5)

    failFlag = 50
    response = []
    #The first stage Open a socket and read a socketID
    while True:
        f = open('/home/bigdata/Desktop/nbiotWithDongle/logs', 'a+')
        try:
            ser.write(bytes("AT+NSOCR=DGRAM,17,8787,1\r\n", 'UTF-8'))
            for _ in range(5):
                response.append(ser.readline().decode("utf-8"))
            f.write("   NSOCR:" + str(response))
            
            if failFlag <= 0:
                #It try more than 50 times reboot the ubuntu
                f.write('Fail Connection/r/n')
                f.close()
                os.system('reboot')
                #Prevent reboot fail
                sys.exit(0)  
            elif 'OK\r\n' in response:
                #response == OK
                #socketID = 1~6       
                socketID = int(response[response.index('OK\r\n') - 2])
                f.write('socketID=' + str(socketID) + '\r\n')
                f.close()
                break
            else:
                failFlag -= 1
                f.write('\r\n')
                f.close()
                continue

        except Exception as e: 
            f.write(str(e))                       

    #Basic sentence for sending message
    SendtoServer = 'AT+NSOST=' + str(socketID) + ',120.126.136.17,5687'
    count = 0
    while True:
        #must be 0 beacause the first package must ping
        if count % 20 == 0:#Ping server every 20 package
            ping(ser)
        f = open('/home/bigdata/Desktop/nbiotWithDongle/logs', 'a+')#open logs
        f.write('Package'+ str(count) + ':')
        count += 1 
        try:            
            message = str(time.time())#now time
            messageHexAscii = ''
            sPort = initialize()#GPS initialize
            lng, lat = getdata(sPort)#GPS data
            #form Message
            message = str(round(lng,5))+ ':' +str(round(lat,5)) + ':' + str(count) + ':' + message 
            #count how many byte
            messageByteNumber = str(len(message.encode('utf-8')))
            #Turn to ascii code
            for x in message:
                messageHexAscii += format(ord(x), "x")
            #Write to Dongle
            if ser.is_open:
                ser.write(bytes(SendtoServer + ',' + messageByteNumber + ','
                        + messageHexAscii + '\r\n', 'UTF-8'))

            #read data 5 line
            response = []
            for _ in range(5):
                read = ser.readline().decode("utf-8")
                if read not None:
                    response.append(read)
            #write message       
            f.write(response.append('successfull\r\n'))
            
        except Exception as e: 
            f.write(str(e) + '\r\n')
            pass

        f.close()
        time.sleep(10)
    ser.close()